In [1]:
# Install required libraries if they are not already installed
import sys
!{sys.executable} -m pip install -r requirements.txt

  Cloning https://github.com/twopirllc/pandas-ta.git (to revision b465491f226d9e07fffd4e59cd0affc9284521ca) to /tmp/pip-install-6m1d0aap/pandas-ta_f388b7676c844b8195433b4970eedd65
  Running command git clone --filter=blob:none --quiet https://github.com/twopirllc/pandas-ta.git /tmp/pip-install-6m1d0aap/pandas-ta_f388b7676c844b8195433b4970eedd65
  Running command git rev-parse -q --verify 'sha^b465491f226d9e07fffd4e59cd0affc9284521ca'
  Running command git fetch -q https://github.com/twopirllc/pandas-ta.git b465491f226d9e07fffd4e59cd0affc9284521ca
  Resolved https://github.com/twopirllc/pandas-ta.git to commit b465491f226d9e07fffd4e59cd0affc9284521ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# Install required libraries
!pip install setuptools
!pip install yfinance
!pip install pandas_ta
!pip install matplotlib
!pip install scikit-learn
!pip install requests

# Import necessary libraries
import requests
import pandas as pd
import pandas_ta as ta
import sqlite3
import os
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the API key (required for economic data)
fredAPIKey = 'f9ac6d2c51f5d5aeea0b07652fd8a52f'

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [28]:
# Verify the Python environment
import sys
print(f"Python executable: {sys.executable}")

# Verify required libraries
required_libraries = [
    'requests', 'pandas', 'pandas_ta', 'sqlite3', 'os', 'yfinance',
    'matplotlib', 'datetime', 'sklearn'
]

for lib in required_libraries:
    try:
        __import__(lib)
        print(f"{lib} is available.")
    except ImportError as e:
        print(f"Error: {lib} is not installed. Ensure it's included in requirements.txt.")

Python executable: /home/rf599/FINALPROJECT/project-env/bin/python
requests is available.
pandas is available.
pandas_ta is available.
sqlite3 is available.
os is available.
yfinance is available.
matplotlib is available.
datetime is available.
sklearn is available.


In [4]:
def resetDatabase(databasePath):

    if os.path.exists(databasePath):
        os.remove(databasePath)
        print(f"Database at '{databasePath}' has been deleted.")
    else:
        print(f"No database found at '{databasePath}'. You can create a new database.")

In [5]:
# Reset the database:
resetDatabase('stonks.db')

No database found at 'stonks.db'. You can create a new database.


In [6]:
def createDatabase(databasePath):
    
    # Establish database connection
    conn = sqlite3.connect(databasePath)
    cursor = conn.cursor()

    # Create the stock_prices table
    cursor.execute("""
        CREATE TABLE stock_prices (
            date DATE NOT NULL,
            symbol TEXT NOT NULL,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            adj_close REAL,
            volume INTEGER,
            PRIMARY KEY (date, symbol)
        )
    """)

    # Create the technical_indicators table
    cursor.execute("""
        CREATE TABLE technical_indicators (
            date DATE NOT NULL,
            symbol TEXT NOT NULL,
            EMA_200 REAL,
            EMA_100 REAL,
            EMA_50 REAL,
            EMA_25 REAL,
            MACD REAL,
            Signal_Line REAL,
            RSI_14 REAL,
            PRIMARY KEY (date, symbol)
        )
    """)

    # Create the economic_indicators table
    cursor.execute("""
        CREATE TABLE economic_indicators (
            date DATE NOT NULL PRIMARY KEY,
            CPI REAL,
            interest_rate REAL
        )
    """)

    # Commit changes and close connection
    conn.commit()
    conn.close()
    print("Database created successfully.")

In [7]:
# Create a fresh database:
createDatabase('stonks.db')

Database created successfully.


In [8]:
def fetchStockData(symbol):
    
    # Fetch data from Yahoo Finance
    data = yf.download(symbol, period='max', auto_adjust=False)
    data.reset_index(inplace=True)

    # Flatten MultiIndex columns if needed
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0].lower() if col[0] != 'Date' else 'date' for col in data.columns]
    
    # Remove any symbol suffixes from column names
    data.columns = [col.split('_')[0] if '_' in col else col for col in data.columns]

    # Rename columns to standardized names
    columnMapping = {
        'adj close': 'adj_close',
        'close': 'close',
        'high': 'high',
        'low': 'low',
        'open': 'open',
        'volume': 'volume',
        'date': 'date'
    }
    data.rename(columns=columnMapping, inplace=True)

    # Check for missing columns and raise an error if any are missing
    missingColumns = set(columnMapping.values()) - set(data.columns)
    if missingColumns:
        raise ValueError(f"Missing expected columns: {missingColumns}")

    # Convert 'date' to datetime and ensure numeric types for other columns
    data['date'] = pd.to_datetime(data['date'], errors='coerce')
    for col in ['adj_close', 'close', 'high', 'low', 'open', 'volume']:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')

    print(f"{symbol} DataFrame Sample:\n")
    print(data.head())
    print(data.tail())
    print("\n")

    return data

In [9]:
def calculateTechnicalIndicators(df):
 
    # Calculate Exponential Moving Averages (EMAs)
    df['EMA_200'] = df['adj_close'].ewm(span=200, adjust=False).mean()
    df['EMA_100'] = df['adj_close'].ewm(span=100, adjust=False).mean()
    df['EMA_50'] = df['adj_close'].ewm(span=50, adjust=False).mean()
    df['EMA_25'] = df['adj_close'].ewm(span=25, adjust=False).mean()

    # Calculate MACD (Moving Average Convergence Divergence) and Signal Line
    shortEMA = df['adj_close'].ewm(span=12, adjust=False).mean()
    longEMA = df['adj_close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = shortEMA - longEMA
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Calculate RSI (Relative Strength Index) with a 14-day window
    df['RSI_14'] = ta.rsi(df['adj_close'], length=14)

    print("Technical Indicators Sample:\n")
    print(df[['date', 'EMA_200', 'EMA_100', 'EMA_50', 'EMA_25', 'MACD', 'Signal_Line', 'RSI_14']].head())
    print("\n")

    return df

In [11]:
def storeAndCleanStockData(symbol, df, databasePath):

    # Ensure 'date' column is in datetime format
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    if df['date'].isnull().any():
        print("Warning: Dropping rows with invalid dates...")
        df = df.dropna(subset=['date'])

    conn = sqlite3.connect(databasePath)
    cursor = conn.cursor()

    try:
        # Check for required columns
        expectedColumns = {'date', 'adj_close', 'close', 'high', 'low', 'open', 'volume',
                           'EMA_200', 'EMA_100', 'EMA_50', 'EMA_25', 'MACD', 'Signal_Line', 'RSI_14'}
        missingColumns = expectedColumns - set(df.columns)
        if missingColumns:
            raise ValueError(f"Missing expected columns in DataFrame: {missingColumns}")

        # Insert stock_prices data
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT OR IGNORE INTO stock_prices (date, symbol, open, high, low, close, adj_close, volume)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (row['date'].strftime('%Y-%m-%d'), symbol, row['open'], row['high'], 
                  row['low'], row['close'], row['adj_close'], row['volume']))

        # Insert technical_indicators data
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT OR IGNORE INTO technical_indicators (date, symbol, EMA_200, EMA_100, EMA_50, EMA_25, MACD, Signal_Line, RSI_14)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (row['date'].strftime('%Y-%m-%d'), symbol, row['EMA_200'], row['EMA_100'], 
                  row['EMA_50'], row['EMA_25'], row['MACD'], row['Signal_Line'], row['RSI_14']))

        # Remove duplicate entries from stock_prices
        cursor.execute("""
            DELETE FROM stock_prices
            WHERE rowid NOT IN (
                SELECT MIN(rowid)
                FROM stock_prices
                GROUP BY date, symbol
            );
        """)

        # Remove duplicate entries from technical_indicators
        cursor.execute("""
            DELETE FROM technical_indicators
            WHERE rowid NOT IN (
                SELECT MIN(rowid)
                FROM technical_indicators
                GROUP BY date, symbol
            );
        """)

        conn.commit()
        print(f"Data for {symbol} successfully stored and cleaned.")

    except Exception as e:
        print(f"An error occurred while storing data: {e}")

    finally:
        conn.close()
        print("Database connection closed.")

In [12]:
def processStockData(symbol, databasePath):

    try:
        # Step 1: Fetch Stock Data
        stockData = fetchStockData(symbol)

        # Step 2: Calculate Technical Indicators
        stockData = calculateTechnicalIndicators(stockData)

        # Step 3: Store Data in Database
        storeAndCleanStockData(symbol, stockData, databasePath)

    except Exception as e:
        print(f"An error occurred while processing {symbol}: {e}")

    finally:
        print("Process completed.")

In [14]:
processStockData('PLTR', 'stonks.db')

[*********************100%***********************]  1 of 1 completed


PLTR DataFrame Sample:

        date  adj_close  close   high   low   open     volume
0 2020-09-30       9.50   9.50  11.41  9.11  10.00  338584400
1 2020-10-01       9.46   9.46  10.10  9.23   9.69  124297600
2 2020-10-02       9.20   9.20   9.28  8.94   9.06   55018300
3 2020-10-05       9.03   9.03   9.49  8.92   9.43   36316900
4 2020-10-06       9.90   9.90  10.18  8.90   9.04   90864000
           date  adj_close      close       high        low       open  \
1050 2024-12-03  70.959999  70.959999  71.370003  66.150002  66.410004   
1051 2024-12-04  69.849998  69.849998  71.180000  67.279999  71.129997   
1052 2024-12-05  71.870003  71.870003  72.980003  69.889999  70.110001   
1053 2024-12-06  76.339996  76.339996  76.820000  72.279999  72.949997   
1054 2024-12-09  72.555000  72.555000  80.910004  71.050003  80.580002   

         volume  
1050  100751400  
1051   86284800  
1052   66585800  
1053   92566000  
1054  124977208  


Technical Indicators Sample:

        date   EMA_

In [15]:
def fetchEconomicData(apiKey):

    baseURL = "https://api.stlouisfed.org/fred/series/observations"
    indicators = {
        "CPI": "CPIAUCSL",          
        "interestRate": "FEDFUNDS"
    }
    
    # List to store combined data
    allData = []

    # Fetch data for each indicator
    for indicatorName, seriesId in indicators.items():
        print(f"Fetching data for {indicatorName} ({seriesId})...")
        params = {
            "series_id": seriesId,
            "api_key": apiKey,
            "file_type": "json"
        }
        response = requests.get(baseURL, params=params)
        
        if response.status_code == 200: # API Key is valid
            data = response.json().get("observations", [])
            for entry in data:
                allData.append({
                    "date": entry["date"],
                    indicatorName: float(entry["value"]) if entry["value"] != "." else None
                })
        else:
            print(f"Error fetching data for {indicatorName}: {response.status_code} - {response.text}")

    # Combine all indicators into a single DataFrame
    df = pd.DataFrame(allData)
    
    # Merge rows with the same date and format 'date' column
    df = df.groupby("date", as_index=False).mean()
    df["date"] = pd.to_datetime(df["date"], errors='coerce')

    print("CPI and Interest Rate data successfully fetched.")

    print("Economic DataFrame Sample:\n")
    print(df.head())
    print(df.tail())
    print("\n")

    return df

In [16]:
economicData = fetchEconomicData(fredAPIKey)

Fetching data for CPI (CPIAUCSL)...
Fetching data for interestRate (FEDFUNDS)...
CPI and Interest Rate data successfully fetched.
Economic DataFrame Sample:

        date    CPI  interestRate
0 1947-01-01  21.48           NaN
1 1947-02-01  21.62           NaN
2 1947-03-01  22.00           NaN
3 1947-04-01  22.00           NaN
4 1947-05-01  21.95           NaN
          date      CPI  interestRate
930 2024-07-01  313.534          5.33
931 2024-08-01  314.121          5.33
932 2024-09-01  314.686          5.13
933 2024-10-01  315.454          4.83
934 2024-11-01      NaN          4.64




In [17]:
def storeEconomicData(economicData, databasePath):

    try:
        # Ensure 'date' is in datetime format
        economicData['date'] = pd.to_datetime(economicData['date'], errors='coerce')
        
        # Fill missing values using forward fill for CPI and interestRate
        economicData['CPI'] = economicData['CPI'].ffill()
        economicData['interestRate'] = economicData['interestRate'].ffill()

        # Drop rows where both CPI and interestRate are missing
        economicData = economicData.dropna(subset=['CPI', 'interestRate'], how='all').reset_index(drop=True)

        # Convert 'date' to string format for database compatibility
        economicData['date'] = economicData['date'].dt.strftime('%Y-%m-%d')

        # Connect to the database
        conn = sqlite3.connect(databasePath)
        cursor = conn.cursor()

        # Insert data into the database
        for _, row in economicData.iterrows():
            cursor.execute("""
                INSERT OR IGNORE INTO economic_indicators (date, CPI, interest_rate)
                VALUES (?, ?, ?)
            """, (row['date'], row['CPI'], row['interestRate']))

        conn.commit()
        print("Economic data successfully cleaned and stored.")

    except Exception as e:
        print(f"An error occurred while storing economic data: {e}")

    finally:
        # Close the database connection
        if 'conn' in locals():
            conn.close()
        print("Database connection closed.\n")

In [18]:
# Store Economic data in the database
storeEconomicData(economicData, 'stonks.db')

Economic data successfully cleaned and stored.
Database connection closed.



In [19]:
def validateStockData(symbol, databasePath):

    conn = None
    try:
        conn = sqlite3.connect(databasePath)
        cursor = conn.cursor()

        # Check stock_prices
        stockQuery = f"""
            SELECT date, symbol, open, high, low, close, adj_close, volume
            FROM stock_prices
            WHERE symbol = '{symbol}'
            ORDER BY date ASC
        """
        stockDf = pd.read_sql(stockQuery, conn)
        print(f"Stock Prices Sample for {symbol}:")
        print(stockDf.head())
        print("\n")

        # Check technical_indicators
        indicatorsQuery = f"""
            SELECT date, symbol, EMA_200, EMA_100, EMA_50, EMA_25, MACD, Signal_Line, RSI_14
            FROM technical_indicators
            WHERE symbol = '{symbol}'
            ORDER BY date ASC
        """
        indicatorsDf = pd.read_sql(indicatorsQuery, conn)
        print(f"Technical Indicators Sample for {symbol}:")
        print(indicatorsDf.head())
        print("\n")
        
        # Check for missing values
        print("\nMissing Values in stock_prices Table:")
        print(stockDf.isnull().sum())
        print("\n")
        
        print("\nMissing Values in technical_indicators Table:")
        print(indicatorsDf.isnull().sum())
        print("\n")

        # Check for duplicates
        stockDuplicates = cursor.execute(f"""
            SELECT date, symbol, COUNT(*) as duplicate_count
            FROM stock_prices
            WHERE symbol = '{symbol}'
            GROUP BY date, symbol
            HAVING duplicate_count > 1
        """).fetchall()

        indicatorsDuplicates = cursor.execute(f"""
            SELECT date, symbol, COUNT(*) as duplicate_count
            FROM technical_indicators
            WHERE symbol = '{symbol}'
            GROUP BY date, symbol
            HAVING duplicate_count > 1
        """).fetchall()

        print("\nDuplicate Entries in stock_prices Table:")
        print(stockDuplicates if stockDuplicates else "No duplicates found.")

        print("\nDuplicate Entries in technical_indicators Table:")
        print(indicatorsDuplicates if indicatorsDuplicates else "No duplicates found.")

    except Exception as e:
        print(f"An error occurred during validation: {e}")

    finally:
        if conn:
            conn.close()
        print("Validation completed and database connection closed.")

In [20]:
# Make sure stock and economic data were successfully stored in the database
validateStockData('PLTR', 'stonks.db')

Stock Prices Sample for PLTR:
         date symbol   open   high   low  close  adj_close     volume
0  2020-09-30   PLTR  10.00  11.41  9.11   9.50       9.50  338584400
1  2020-10-01   PLTR   9.69  10.10  9.23   9.46       9.46  124297600
2  2020-10-02   PLTR   9.06   9.28  8.94   9.20       9.20   55018300
3  2020-10-05   PLTR   9.43   9.49  8.92   9.03       9.03   36316900
4  2020-10-06   PLTR   9.04  10.18  8.90   9.90       9.90   90864000


Technical Indicators Sample for PLTR:
         date symbol   EMA_200   EMA_100    EMA_50    EMA_25      MACD  \
0  2020-09-30   PLTR  9.500000  9.500000  9.500000  9.500000  0.000000   
1  2020-10-01   PLTR  9.499602  9.499208  9.498431  9.496923 -0.003191   
2  2020-10-02   PLTR  9.496621  9.493283  9.486728  9.474083 -0.026395   
3  2020-10-05   PLTR  9.491978  9.484109  9.468817  9.439923 -0.057836   
4  2020-10-06   PLTR  9.496038  9.492345  9.485726  9.475313 -0.012408   

   Signal_Line  RSI_14  
0     0.000000     NaN  
1    -0.000638 

In [22]:
def predictStockPrice(symbol, futureDays, databasePath):

    # Connect to the database
    conn = sqlite3.connect(databasePath)

    # Query stock prices and technical indicators
    query = f"""
        SELECT sp.date, sp.adj_close, sp.open, sp.high, sp.low, sp.close, sp.volume,
               ti.EMA_200, ti.EMA_100, ti.EMA_50, ti.EMA_25, ti.MACD, ti.Signal_Line, ti.RSI_14,
               sp.symbol
        FROM stock_prices sp
        JOIN technical_indicators ti ON sp.date = ti.date AND sp.symbol = ti.symbol
        WHERE sp.symbol = '{symbol}'
        ORDER BY sp.date ASC
    """
    df = pd.read_sql(query, conn)
    conn.close()

    # Debugging: Check DataFrame structure
    print("DataFrame Columns:", df.columns)

    # Ensure all required columns are present
    requiredColumns = {'date', 'adj_close', 'symbol', 'EMA_200', 'EMA_100', 'EMA_50', 'EMA_25', 'MACD', 'Signal_Line', 'RSI_14'}
    missingColumns = requiredColumns - set(df.columns)
    if missingColumns:
        raise ValueError(f"Missing required columns: {missingColumns}")

    # Ensure 'date' is in datetime format
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # Create target column for future price prediction
    df['target'] = df['adj_close'].shift(-futureDays)

    # Drop rows with missing target
    df = df.dropna(subset=['target'])

    # Prepare features (X) and target (y)
    X = df.drop(columns=['target', 'date', 'symbol', 'adj_close'])
    y = df['target']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

    # Train a Random Forest Regressor
    model = RandomForestRegressor(n_estimators=100, random_state=17)
    model.fit(X_train, y_train)

    # Evaluate model performance
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"\nModel Performance:\nMSE: {mse:.2f}, MAE: {mae:.2f}, R2: {r2:.2f}")

    # Predict the price for the most recent data
    latestData = X.iloc[-1:]
    futurePrice = model.predict(latestData)[0]
    print(f"Predicted price {futureDays} days ahead: ${futurePrice:.2f}")

    return model, mse, mae, r2, futurePrice

In [23]:
predictStockPrice('PLTR', 30, 'stonks.db')

DataFrame Columns: Index(['date', 'adj_close', 'open', 'high', 'low', 'close', 'volume',
       'EMA_200', 'EMA_100', 'EMA_50', 'EMA_25', 'MACD', 'Signal_Line',
       'RSI_14', 'symbol'],
      dtype='object')

Model Performance:
MSE: 3.09, MAE: 0.91, R2: 0.96
Predicted price 30 days ahead: $72.37


(RandomForestRegressor(random_state=17),
 3.087124529546909,
 0.9137005254233758,
 0.9635694245556993,
 72.37099960327149)

In [26]:
def plotStockData(symbol, databasePath, days=None):

    # Connect to the database
    conn = sqlite3.connect(databasePath)

    try:
        # Query stock prices and technical indicators
        query = f"""
            SELECT sp.date, sp.adj_close AS close, 
                   ti.EMA_200, ti.EMA_100, ti.EMA_50, ti.EMA_25, 
                   ti.MACD, ti.Signal_Line, ti.RSI_14
            FROM stock_prices sp
            JOIN technical_indicators ti ON sp.date = ti.date AND sp.symbol = ti.symbol
            WHERE sp.symbol = '{symbol}'
            ORDER BY sp.date ASC
        """
        df = pd.read_sql(query, conn)
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)

        # Use all data if days parameter is provided
        if days is not None:
            most_recent_date = df.index.max()
            start_date = most_recent_date - timedelta(days=days)
            df = df[df.index >= start_date]

        # Create the figure with three subplots
        plt.figure(figsize=(17, 20))

        # Share price and EMA lines
        plt.subplot(3, 1, 1)
        plt.plot(df.index, df['close'], label='Close Price', linewidth=1.5)
        plt.plot(df.index, df['EMA_200'], label='200-Day EMA', linestyle='--', linewidth=.5)
        plt.plot(df.index, df['EMA_100'], label='100-Day EMA', linestyle='--', linewidth=.5)
        plt.plot(df.index, df['EMA_50'], label='50-Day EMA', linestyle='--', linewidth=.5)
        plt.plot(df.index, df['EMA_25'], label='25-Day EMA', linestyle='--', linewidth=.5)
        plt.title(f"Stock Price and EMA Lines for {symbol}")
        plt.ylabel("Price")
        plt.legend()
        plt.grid()

        # MACD and Signal Line
        plt.subplot(3, 1, 2)
        plt.plot(df.index, df['MACD'], label='MACD', color='blue', linewidth=1.5)
        plt.plot(df.index, df['Signal_Line'], label='Signal Line', color='red', linestyle='--')
        plt.title(f"MACD and Signal Line for {symbol}")
        plt.ylabel("Value")
        plt.legend()
        plt.grid()

        # RSI with overbought and oversold lines
        plt.subplot(3, 1, 3)
        plt.plot(df.index, df['RSI_14'], label='RSI', color='red', linewidth=1.5)
        plt.axhline(75, color='red', linestyle='--', label='Overbought (75)', linewidth=1)
        plt.axhline(25, color='green', linestyle='--', label='Oversold (25)', linewidth=1)
        plt.ylim(0, 100)
        plt.title(f"RSI for {symbol}")
        plt.ylabel("RSI Value")
        plt.legend()
        plt.grid()

        plt.show()

    finally:
        conn.close()

In [ ]:
plotStockData('PLTR', 'stonks.db', 365)